In [49]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 langchain-google-vertexai \
                                 langchain pypdf faiss-gpu chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.2 MB

In [37]:
import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [38]:
# Define project information
PROJECT_ID = "crime-information-421611"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [39]:
from IPython.display import Markdown
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_vertexai import ChatVertexAI, HarmBlockThreshold, HarmCategory
from vertexai.generative_models import Content, GenerativeModel, Part
from pypdf import PdfReader

import os
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.embeddings import VertexAIEmbeddings

In [40]:
model = GenerativeModel("gemini-1.0-pro")

In [41]:
pdf_docs = [
    "/content/2022-2023-Annual-Crime-Statistics-Report.pdf",
    "/content/2023-2024 _Annual_WEB.pdf",
    "/content/2023-2024_-_2nd_Quarter_WEB.pdf",
    "/content/2023-2024_-_3nd_Quarter_WEB.pdf",
    "/content/april_june_2023_24_quarter1_presentation.pdf"
]

def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [42]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        chunk_size=10000,
        chunk_overlap=1000
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [43]:
def get_vector_store(text_chunks):
    embeddings = VertexAIEmbeddings(model="models/embedding-001")
    vector_store = Chroma.from_texts(texts=text_chunks, embedding=embeddings)
    vector_store.save_local("Chroma_index")

In [44]:
def get_conversation_chain():
    prompt_template = """
        Answer the question clear and precise. If not provided the context return the result as
        "Sorry I dont know the answer", don't provide the wrong answer.
        Context:\n {context}?\n
        Question:\n{question}\n
        Answer:
    """
    model = ChatVertexAI(
      model_name="gemini-1.0-pro",
      safety_settings={
          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
      },
    )
    prompt = ChatPromptTemplate(template=prompt_template, input_variables=['context', 'question'])
    memory = ConversationBufferMemory(memory_key="history", return_messages=True)
    conversation = ConversationChain(llm=model, prompt=prompt, verbose=True, memory=memory)

    return conversation

In [51]:
def user_input(query, k=2):
    file = "/content/2022-2023-Annual-Crime-Statistics-Report.pdf"
    loader = PyPDFLoader(file)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    embeddings = VertexAIEmbeddings(model='models/embedding-001')
    db = Chroma.from_documents(texts, embeddings)
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # new_db = Chroma.load_local("Chroma_index", embeddings)
    # docs = new_db.similarity_search(user_question)

    chain = get_conversation_chain()
    response = chain.invoke(
        {"input_documents": retriever, "question": query},
        return_only_outputs=True
    )

    return response["output_text"]

In [52]:
print(user_input("What is the crime rate in the city of Sandton?"))

KeyError: 'messages'